In [2]:
import os
import re

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid

In [3]:
# Load 2021 CSD boundaries (GTA subset source)
gdf_gta_csd = gpd.read_file('../data/geo/regions/GTA_CSD.gpkg').to_crs(4326)

# Load former Toronto municipalities
gdf_oldto = gpd.read_file(
    '../data/geo/regions/Former Municipality Boundaries Data - 4326.geojson'
).to_crs(4326)

In [4]:
MUNICIPALITIES = [
    'Toronto',
    'Mississauga',
    'Brampton',
    'Vaughan',
    'Richmond Hill',
    'Markham',
    'Pickering',
    'Ajax'
]

# Subset to TMUN municipalities
gdf_tmun_csd = gdf_gta_csd[gdf_gta_csd['CSDNAME'].isin(MUNICIPALITIES)]

# Save TMUN_CSD (NO union)
gdf_tmun_csd.to_file('../data/geo/regions/TMUN_CSD.gpkg', driver='GPKG')

# Create union geometry for TMUN
tmun_union_geom = make_valid(gdf_tmun_csd.unary_union)

# Create GeoDataFrame wrapper
gdf_tmun_union = gpd.GeoDataFrame(
    {'name': ['TMUN']},
    geometry=[tmun_union_geom],
    crs=gdf_tmun_csd.crs
)

# Save union file
gdf_tmun_union.to_file('../data/geo/regions/TMUN.gpkg', driver='GPKG')


In [5]:
# Extract modern Toronto boundary from 2021 CSD file
gdf_toronto = gdf_gta_csd[gdf_gta_csd['CSDNAME'] == 'Toronto'].copy()
toronto_geom = make_valid(gdf_toronto.unary_union)

# Clean up old Toronto polygons (ensure validity)
gdf_oldto['geometry'] = gdf_oldto['geometry'].apply(make_valid)

# Clip each former municipality to the modern Toronto boundary
gdf_oldto_clipped = gdf_oldto.copy()
gdf_oldto_clipped['geometry'] = gdf_oldto_clipped.geometry.intersection(toronto_geom)

# Remove empty geometries (e.g. due to mismatch or gaps)
gdf_oldto_clipped = gdf_oldto_clipped[~gdf_oldto_clipped.geometry.is_empty]

# Rename for clarity
gdf_oldto_clipped = gdf_oldto_clipped[['AREA_NAME', 'geometry']]
gdf_oldto_clipped = gdf_oldto_clipped.rename(columns={'AREA_NAME': 'OLD_MUN'})

In [6]:
# Remove Toronto from TMUN_CSD
gdf_except_toronto = gdf_tmun_csd[gdf_tmun_csd['CSDNAME'] != 'Toronto'].copy()

# Harmonize columns for concatenation
gdf_except_toronto = gdf_except_toronto[['CSDNAME', 'geometry']]
gdf_oldto_clipped = gdf_oldto_clipped[['OLD_MUN', 'geometry']]

# For consistent column names
gdf_except_toronto = gdf_except_toronto.rename(columns={'CSDNAME': 'NAME'})
gdf_oldto_clipped = gdf_oldto_clipped.rename(columns={'OLD_MUN': 'NAME'})

# Combine
gdf_tmun_csd_oldto = pd.concat([
    gdf_except_toronto,
    gdf_oldto_clipped
], ignore_index=True)

# Save
gdf_tmun_csd_oldto.to_file(
    '../data/geo/regions/TMUN_CSD_OldTO.gpkg',
    driver='GPKG'
)

In [ ]:
MUNICIPALITIES = ['Toronto', 'Mississauga', 'Brampton', 'Vaughan', 'Richmond Hill', 'Markham', 'Pickering', 'Ajax']

gdf_gta_csd = gpd.read_file('../data/geo/regions/GTA_CSD.gpkg').to_crs(4326)
gdf_tmun_csd = gdf_gta_csd[gdf_gta_csd['CSDNAME'].isin(MUNICIPALITIES)]

# Create union geometry
tmun_union_geom = gdf_tmun_csd.unary_union

# Create GeoDataFrame for the union with proper validation
gdf_tmun_union_geom = gpd.GeoDataFrame(
    {'name': ['Target_Municipalities']}, 
    geometry=[make_valid(tmun_union_geom)], 
    crs=gdf_gta_csd.crs
)

# Save to file
os.makedirs('../data/geo/regions', exist_ok=True)
gdf_tmun_union_geom.to_file('../data/geo/regions/TMUN.gpkg', driver='GPKG')

# Keep the original variable for backward compatibility
gdf_tmun_union = tmun_union_geom